In [20]:
import pandas as pd 
import os
import json

In [21]:
cwd = os.getcwd()
print(cwd)

c:\Users\t.pelser\repos\ethos.REFLOW\example_workflows\offshore_north_sea


In [22]:
# Define the file paths for the two scenarios
file_paths = {
    '1000m_depth': os.path.join(os.getcwd(), 'output/geodata/turbine_placements_4326_1000m_depth.csv'),
    '50m_depth': os.path.join(os.getcwd(), 'output/geodata/turbine_placements_4326_50m_depth.csv')
}

In [23]:
report_paths = {
    '1000m_depth': os.path.join(os.getcwd(), 'output/report_1000m_depth.json'),
    '50m_depth': os.path.join(os.getcwd(), 'output/report_50m_depth.json')
}

In [24]:
# Define the turbine capacity in MW
turbine_capacity_mw = 15

In [25]:
# Function to calculate the statistics for a given scenario
def calculate_statistics(file_path, report_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Count the number of turbines installed in each country
    turbine_counts = df['country'].value_counts()
    
    # Calculate the total installable capacity for each country in GW
    installable_capacity_gw = turbine_counts * turbine_capacity_mw / 1000  # Convert MW to GW
    
    # Create a DataFrame to display the results
    results = pd.DataFrame({
        'Number of Turbines': turbine_counts,
        'Total Installable Capacity (GW)': installable_capacity_gw
    })
    
    # Calculate the totals for the entire North Sea region
    total_turbines_ns = turbine_counts.sum()
    total_installable_capacity_ns_gw = installable_capacity_gw.sum()
    
    # Read the report JSON file
    with open(report_path, 'r') as file:
        report_data = json.load(file)
    
    # Extract generation data and capacity factor for each year
    generation_twh = []
    capacity_factors = []
    for year in range(2013, 2024):
        generation_key = f"Total_Generation_{year}_TWh"
        capacity_factor_key = f"Mean_Capacity_Factor_{year}"
        if generation_key in report_data:
            generation_twh.append(report_data[generation_key])
        if capacity_factor_key in report_data:
            capacity_factors.append(report_data[capacity_factor_key])
    
    # Calculate min, max, and mean generation
    min_generation = min(generation_twh)
    max_generation = max(generation_twh)
    mean_generation = sum(generation_twh) / len(generation_twh)
    
    # Calculate overall mean capacity factor
    overall_mean_capacity_factor = sum(capacity_factors) / len(capacity_factors)
    
    return results, total_turbines_ns, total_installable_capacity_ns_gw, min_generation, max_generation, mean_generation, overall_mean_capacity_factor

In [27]:
# Process each scenario and display the results
for scenario, file_path in file_paths.items():
    report_path = report_paths[scenario]
    print(f"\nStatistics for {scenario.replace('_', ' ').title()} Scenario:")
    
    results, total_turbines_ns, total_installable_capacity_ns_gw, min_generation, max_generation, mean_generation, overall_mean_capacity_factor = calculate_statistics(file_path, report_path)
    
    # Display the results for each country
    print(results)
    
    # Display the totals for the entire North Sea region
    print(f"\nTotal Statistics for the {scenario.replace('_', ' ').title()} Scenario:")
    print(f"Total Number of Turbines: {total_turbines_ns}")
    print(f"Total Installable Capacity: {total_installable_capacity_ns_gw:.2f} GW")
    print(f"Mean Annual Generation: {mean_generation:.2f} TWh")
    print(f"Max Annual Generation: {max_generation:.2f} TWh")
    print(f"Min Annual Generation: {min_generation:.2f} TWh")
    print(f"Overall Mean Capacity Factor: {overall_mean_capacity_factor:.2f}")


Statistics for 1000M Depth Scenario:
         Number of Turbines  Total Installable Capacity (GW)
country                                                     
GBR                   20179                          302.685
NOR                   14435                          216.525
DNK                    7973                          119.595
NLD                    4781                           71.715
DEU                    3260                           48.900
SWE                     411                            6.165
BEL                     129                            1.935
FRA                      44                            0.660

Total Statistics for the 1000M Depth Scenario:
Total Number of Turbines: 51212
Total Installable Capacity: 768.18 GW
Mean Annual Generation: 3314.30 TWh
Max Annual Generation: 3722.24 TWh
Min Annual Generation: 2030.05 TWh
Overall Mean Capacity Factor: 49.25

Statistics for 50M Depth Scenario:
         Number of Turbines  Total Installable Capacity 